In [ ]:
import requests
import bs4 as bs
from datetime import datetime
from datetime import timedelta 
import pandas_datareader.data as web
import os
import pandas as pd
import time
import numpy as np

In [ ]:
def import_sp500():
    req = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S.26P_500_Component_Stocks')
    soup = bs.BeautifulSoup(req.text)
    table = soup.find('table',{'class':'wikitable sortable'})
    symbols = []
    for company in table.findAll('tr')[1:]:
        symbol = company.findAll('td')[0].get_text()
        symbols.append(symbol.strip())
    return symbols

In [ ]:
symbols = []
for x in import_sp500():
    symbols.append(x.replace('.','-'))

In [ ]:
# get stock price from 2000-01-01 and updated daily
def get_stock_price(symbols):
    for symbol in symbols:
        #time.sleep(3)
        if os.path.exists('data/{}.csv'.format(symbol)):
            df = pd.read_csv('data/{}.csv'.format(symbol), usecols = ['Date'])
            last_date = df.iloc[-1]['Date']
            last_date_yymmdd = datetime.strptime(last_date, '%Y-%m-%d')
            new_date = last_date_yymmdd + timedelta(days = 1)
            new_start = datetime(new_date.year, new_date.month, new_date.day)
            # print()
            if new_start < datetime.now():
                df_add = web.DataReader(symbol, 'yahoo', new_start, datetime.now())
                with open('data/{}.csv'.format(symbol), 'a') as f:
                    df_add.to_csv(f, header=False)
        else:
            start = datetime(2000, 1, 1)
            df = web.DataReader(symbol, 'yahoo', start, datetime.now())
            df.to_csv('data/{}.csv'.format(symbol))

In [ ]:
get_stock_price(symbols)